## Setup configuration for our validation

In [1]:
HD_OUTPUT_FILENAME = 'hashdice_validation.txt'
PREVIOUS_TIMESTEP_COUNT = 4
IMPORT_COUNT = 100000 + PREVIOUS_TIMESTEP_COUNT

## Generate deterministic dataset

In [2]:
import hmac
import hashlib

def hashdice(secret_key, message):
    hmac_sha512 = hmac.new(key=secret_key.encode(), msg=message.encode(), digestmod=hashlib.sha512).hexdigest()
    lucky = int(hmac_sha512[0:5], 16)
    return lucky % 100000


with open(HD_OUTPUT_FILENAME, 'w') as f:

    for i in range(IMPORT_COUNT + 1):
        secret_key = '4671f003b18db91be8f1570c7f681ad9e97026c5f34d89a733f48ab81eb8d592'
        message = f'5533idsczGPRYacG5N15jfJ6nm5:{i}'
        result = hashdice(secret_key, message)
        rs = str(result)
        f.write(f'{rs + str(i%10000)}\n')

## Convert to input and target data

In [3]:
import numpy as np

# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
    shp = a.shape
    s  = a.strides
    nd0 = shp[0]-L+1
    shp_in = (nd0,L)+shp[1:]
    strd_in = (s[0],) + s
    return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

df = np.genfromtxt(HD_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)
print(BIT_WIDTH)
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

#indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
#print(indicies)
#np.random.shuffle(indicies)
#print(indicies)
#df_as_frames=df_as_frames[indicies]

# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

print(np.shape(X), np.shape(y))


print(X[0])
print(y[0])

30
(100001, 120) (100001, 30)
[0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0]


## Load pre-trained model

In [4]:
from tensorflow.keras.models import load_model
model = load_model('hashdice_model.h5')

## Validate bitwise accuracy

In [5]:
results = model.evaluate(X, y, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

391/391 [==============================] - 2s 4ms/step - loss: 0.5914 - binary_accuracy: 0.5959
test loss: 0.591414, test acc: 0.5959078073501587


In [6]:
import tensorflow as tf
index = 2
inputTest=np.reshape(X[index],(1,120))
predict = model.predict(inputTest)

1/1 [==============================] - 0s 129ms/step


In [7]:
orignalResult = ', '.join([str(x) for x in predict[0]])
print(f'Original Result {orignalResult}')
orignalResult = ', '.join([str(x) for x in predict[0][::-1]])
print(f'-------------------------')
print(f'Original Result {orignalResult}')
npWhere=np.where(predict[0] > 0.2, 1, 0)
print(f'reverse correct output------------{y[index]}')
print(f'predicted output reverse----------{npWhere}')
o = y[index][::-1]
print(f'correct output--------------------{o}')
npWhere = npWhere[::-1]
print(f'predicted output------------------{npWhere}')
tfround = tf.round(predict[0])
#print(f'tfround {tfround}')

output = sum(bit * 2**(len(o)-i-1) for i, bit in enumerate(o))
print(f'correct output--------------------{output}')
result = np.array(tfround[::-1])
result = sum(bit * 2**(len(result)-i-1) for i, bit in enumerate(result))
print(f'predicted output------------------{result}')

Original Result 2.428592e-08, 0.88184845, 0.9631405, 0.577082, 0.4396599, 0.61575747, 0.4377974, 0.54967254, 0.53978455, 0.39816344, 0.37197503, 0.62312376, 0.55386335, 0.46072546, 0.5343259, 0.34333903, 0.4962835, 0.4796249, 0.55748653, 0.3092719, 0.116490826, 0.13523231, 0.12588662, 0.08605301, 0.006725871, 0.010726309, 0.0013513016, 8.911185e-05, 1.090546e-05, 2.3909619e-05
-------------------------
Original Result 2.3909619e-05, 1.090546e-05, 8.911185e-05, 0.0013513016, 0.010726309, 0.006725871, 0.08605301, 0.12588662, 0.13523231, 0.116490826, 0.3092719, 0.55748653, 0.4796249, 0.4962835, 0.34333903, 0.5343259, 0.46072546, 0.55386335, 0.62312376, 0.37197503, 0.39816344, 0.53978455, 0.54967254, 0.4377974, 0.61575747, 0.4396599, 0.577082, 0.9631405, 0.88184845, 2.428592e-08
reverse correct output------------[0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
predicted output reverse----------[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
correct output--------

In [54]:
#000000000011010100001101001010
#000000000001101001101101001110